In [ ]:
!pip install praw dotenv google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 8.5 MB/s eta 0:00:00


In [ ]:
import praw
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
import os
from dotenv import load_dotenv
from google.colab import drive

# Mount Drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/MyDrive/.env /content

# Setup API

In [ ]:
load_dotenv()
CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
API_KEY = os.getenv("YOUTUBE_API_KEY")

In [ ]:
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent="My_Scrapper"
)

In [ ]:
youtube = build("youtube", "v3", developerKey=API_KEY)

# Reddit Scrapping

In [ ]:
subreddit = reddit.subreddit("marvelstudios")

In [ ]:
comments_data = []
for post in subreddit.search("avengers doomsday", sort="new", time_filter="month", limit=100):
    post.comments.replace_more(limit=0)
    for comment in post.comments.list():
        comments_data.append({"title": post.title, "comment": comment.body})

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [ ]:
df_reddit = pd.DataFrame(comments_data)
df_reddit["source"] = "Reddit"
df_reddit.to_csv("reddit_avengers_doomsday_comments.csv", index=False)
df_reddit.info()
df_reddit.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4780 entries, 0 to 4779
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    4780 non-null   object
 1   comment  4780 non-null   object
 2   source   4780 non-null   object
dtypes: object(3)
memory usage: 112.2+ KB


,title,comment,source
0,Hawkeye in Doomsday?,"Nope, they have to keep the running joke that ...",Reddit
1,Hawkeye in Doomsday?,Kate is going to be in an Avengers movie so gu...,Reddit
2,Hawkeye in Doomsday?,"Mhmm personally i dont want Hawkeye to die, im...",Reddit
3,Hawkeye in Doomsday?,Renner is literally a real life superhero and ...,Reddit
4,Hawkeye in Doomsday?,"If he's not, they lose",Reddit


# Youtube Scrapping

In [ ]:
search_query = "avengers doomsday"
search_response = youtube.search().list(
    q=search_query,
    part="id,snippet",
    maxResults=5,
    type="video"
).execute()
videos = [{"title": item["snippet"]["title"], "videoId": item["id"]["videoId"]} for item in search_response["items"]]

In [ ]:
comments_data = []
for video in videos:
    video_id = video["videoId"]
    video_title = video["title"]

    try:
        response = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100
        ).execute()

        while response:
            for item in response["items"]:
                comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
                comments_data.append({"title": video_title, "comment": comment})

            if "nextPageToken" in response:
                response = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=100,
                    pageToken=response["nextPageToken"]
                ).execute()
            else:
                response = None

    except HttpError as e:
        if e.resp.status == 403 and "commentsDisabled" in str(e):
            print(f"Skipping video {video_id} ({video_title}) - Comments disabled (403 error)")
        else:
            print(f"Error fetching comments for {video_id} ({video_title}): {e}")

In [ ]:
df_youtube = pd.DataFrame(comments_data)
df_youtube["source"] = "YouTube"
df_youtube.to_csv("youtube_avengers_doomsday_comments.csv", index=False)
df_youtube.info()
df_youtube.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24592 entries, 0 to 24591
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    24592 non-null  object
 1   comment  24592 non-null  object
 2   source   24592 non-null  object
dtypes: object(3)
memory usage: 576.5+ KB


,title,comment,source
0,Avengers: Doomsday (2026) - First Trailer | Ma...,Congratulations RDJ is back,YouTube
1,Avengers: Doomsday (2026) - First Trailer | Ma...,Uma grande merda.,YouTube
2,Avengers: Doomsday (2026) - First Trailer | Ma...,Like for this trailer,YouTube
3,Avengers: Doomsday (2026) - First Trailer | Ma...,Help ❤,YouTube
4,Avengers: Doomsday (2026) - First Trailer | Ma...,No reason to watch any more marvel movies.<br>...,YouTube


# Export CSV

In [ ]:
!cp reddit_avengers_doomsday_comments.csv /content/gdrive/MyDrive
!cp youtube_avengers_doomsday_comments.csv /content/gdrive/MyDrive

# Evaluate

*   There's a lot of comments with url, emojis and special characters
*   Reddit comments are could be much longer than youtube
*   Youtube comments have quite a lot of bots and spam comments
*   The comments is mostly english, but there's some other languages as well





